In [24]:
import pickle
import pandas as pd
with open('/Users/alishakhan/Desktop/Career/Ascent Integrated Tech/task1/CubiCasa5k_git/submission/ignore_outputs/data.pkl', 'rb') as f:
    embeddings, Y, df, X_all, edges_df_list, attributes_df_list = pickle.load(f)

with open('/Users/alishakhan/Desktop/Career/Ascent Integrated Tech/task1/CubiCasa5k_git/submission/ignore_outputs/test_data.pkl', 'rb') as f:
    test_embeddings, test_Y, test_df, test_X_all, test_edges_df_list, test_attributes_df_list = pickle.load(f)

Y=Y.argmax(1)
test_Y=test_Y.argmax(1)




import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from torch_geometric.data import DataLoader

room_classes_names = ["Background", "Outdoor", "Wall", "Kitchen", "Living Room" ,"Bed Room", "Bath", "Entry", "Railing", "Storage", "Garage"]

for df in attributes_df_list:
    df["Room Type"] = df["Room Type"].apply(lambda x: room_classes_names.index(x))

# Define a function to create a PyTorch Geometric Data object from the room and edge data for each floorplan
def create_data(attributes_df, edges_df):
    edge_index = edges_df[['source', 'target']].values.T
    x = attributes_df[['Area', 'Relative Area', 'Number of neighboring rooms']].values
    y = attributes_df['Room Type'].values
    return Data(x=torch.tensor(x, dtype=torch.float), edge_index=torch.tensor(edge_index, dtype=torch.long), y=torch.tensor(y, dtype=torch.long))

# Create a list of Data objects, one for each floorplan
data_list = []
for i in range(len(attributes_df_list)):
    attributes_df_i = attributes_df_list[i]
    edges_df_i = edges_df_list[i]
    data_list.append(create_data(attributes_df_i, edges_df_i))

# Define a function to create a PyTorch Geometric Data object from the room and edge data for each floorplan
def create_data(attributes_df, edges_df):
    edge_index = edges_df[['source', 'target']].values.T
    x = attributes_df[['Area', 'Relative Area', 'Number of neighboring rooms']].values
    return Data(x=torch.tensor(x, dtype=torch.float), edge_index=torch.tensor(edge_index, dtype=torch.long), y=None)

# Create a list of Data objects for the test floorplans
test_data_list = []
for i in range(len(test_attributes_df_list)):
    attributes_df_i = test_attributes_df_list[i].drop('Room Type', axis=1)
    edges_df_i = test_edges_df_list[i]
    test_data_list.append(create_data(attributes_df_i, edges_df_i))
    
# Define the GCN architecture
class Net(torch.nn.Module):
    def __init__(self, num_features, num_classes):
        super(Net, self).__init__()
        self.conv1 = GCNConv(num_features, 16)
        self.conv2 = GCNConv(16, num_classes)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Create the GCN and optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(num_features=3, num_classes=len(room_classes_names)).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

# Define the training loop
def train(model, optimizer, data_loader):
    model.train()
    total_loss = 0
    for data in data_loader:
        optimizer.zero_grad()
        out = model(data.x.to(device), data.edge_index.to(device))
        loss = F.nll_loss(out, data.y.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * data.num_graphs
    return total_loss / len(data_loader.dataset)

# Train the GCN on your floorplan data
num_classes = len(room_classes_names)
model = Net(num_features=3, num_classes=len(room_classes_names)).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
data_loader = DataLoader(data_list, batch_size=32, shuffle=True)
for epoch in range(200):
    loss = train(model, optimizer, data_loader)
    #print('Epoch: {:03d}, Loss: {:.5f}'.format(epoch, loss))
    
# Add the hyperparameters in the Net class
class Net(torch.nn.Module):
    def __init__(self, num_features, num_classes, hidden_units, dropout_rate):
        super(Net, self).__init__()
        self.conv1 = GCNConv(num_features, hidden_units)
        self.conv2 = GCNConv(hidden_units, num_classes)
        self.dropout_rate = dropout_rate

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=self.dropout_rate, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)
    
    
# Define a list of hyperparameters to search over
hyperparameters = {
    'hidden_units': [8, 16, 32],
    'dropout_rate': [0.3, 0.5, 0.7],
    'learning_rate': [0.001, 0.01, 0.1],
    'weight_decay': [1e-4, 5e-4, 1e-3],
    'batch_size': [16, 32, 64],
    'num_epochs': [100, 200, 300]
}


# Define a function to perform grid search
def grid_search(hyperparameters):
    best_accuracy = 0
    best_hyperparams = None

    # Iterate over all possible combinations of hyperparameters
    for hidden_units in hyperparameters['hidden_units']:
        for dropout_rate in hyperparameters['dropout_rate']:
            for learning_rate in hyperparameters['learning_rate']:
                for weight_decay in hyperparameters['weight_decay']:
                    for batch_size in hyperparameters['batch_size']:
                        for num_epochs in hyperparameters['num_epochs']:
                            print(f"hidden units {hidden_units}")
                            print(f"dropout rate {dropout_rate}")
                            print(f"learning rate {learning_rate}")
                            print(f"weight decay {weight_decay}")
                            print(f"batch size {batch_size}")
                            print(f"num epochs {num_epochs}")
                            # Train the model using the current set of hyperparameters
                            model = Net(num_features=3, num_classes=len(room_classes_names), hidden_units=hidden_units, dropout_rate=dropout_rate).to(device)
                            optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
                            data_loader = DataLoader(data_list, batch_size=batch_size, shuffle=True)
                            for epoch in range(num_epochs):
                                loss = train(model, optimizer, data_loader)

                            # Evaluate the model on the test data
                            test_data_loader = DataLoader(test_data_list, batch_size=batch_size, shuffle=False)
                            model.eval()
                            predictions = []
                            with torch.no_grad():
                                for data in test_data_loader:
                                    logits = model(data.x.to(device), data.edge_index.to(device))
                                    pred = logits.argmax(dim=1)
                                    pred_room_types = [room_classes_names[i] for i in pred.tolist()]
                                    pred_room_types = pred_room_types + [None] * (len(data.x) - len(pred_room_types))
                                    predictions.extend(pred_room_types)

                            # Create a list of actual room types for the test data
                            actual_room_types = []
                            for attributes_df_i in test_attributes_df_list:
                                actual_room_types.extend(attributes_df_i['Room Type'].tolist())

                            # Compare the actual and predicted room types and calculate accuracy
                            correct_predictions = 0
                            for i in range(len(actual_room_types)):
                                if actual_room_types[i] == predictions[i]:
                                    correct_predictions += 1
                            accuracy = correct_predictions / len(actual_room_types)

                            # Check if the current set of hyperparameters yields a better accuracy
                            if accuracy > best_accuracy:
                                best_accuracy = accuracy
                                best_hyperparams = {
                                    'hidden_units': hidden_units,
                                    'dropout_rate': dropout_rate,
                                    'learning_rate': learning_rate,
                                    'weight_decay': weight_decay,
                                    'batch_size': batch_size,
                                    'num_epochs': num_epochs
                                }

    return best_hyperparams

# Perform grid search
best_hyperparams = grid_search(hyperparameters)
print('Best hyperparameters found:', best_hyperparams)

# Train the model with the best hyperparameters
model = Net(num_features=3, num_classes=len(room_classes_names), hidden_units=best_hyperparams['hidden_units'], dropout_rate=best_hyperparams['dropout_rate']).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=best_hyperparams['learning_rate'], weight_decay=best_hyperparams['weight_decay'])
data_loader = DataLoader(data_list, batch_size=best_hyperparams['batch_size'], shuffle=True)
for epoch in range(best_hyperparams['num_epochs']):
    loss = train(model, optimizer, data_loader)
    print('Epoch: {:03d}, Loss: {:.5f}'.format(epoch, loss))

# Use the trained model to predict room types for the test data
test_data_loader = DataLoader(test_data_list, batch_size=32, shuffle=False)
model.eval()
predictions = []
with torch.no_grad():
    for data in test_data_loader:
        logits = model(data.x.to(device), data.edge_index.to(device))
        pred = logits.argmax(dim=1)
        pred_room_types = [room_classes_names[i] for i in pred.tolist()]
        # Create a list of None values with the same length as the number of nodes in the graph
        pred_room_types = pred_room_types + [None]*(len(data.x) - len(pred_room_types))
        predictions.extend(pred_room_types)

# Create a list of actual room types for the test data
actual_room_types = []
for attributes_df_i in test_attributes_df_list:
    actual_room_types.extend(attributes_df_i['Room Type'].tolist())

# Compare the actual and predicted room types and calculate accuracy
correct_predictions = 0
for i in range(len(actual_room_types)):
    if actual_room_types[i] == predictions[i]:
        correct_predictions += 1
accuracy = correct_predictions / len(actual_room_types)

print('Accuracy: {:.2f}%'.format(accuracy * 100))




/Users/alishakhan/opt/miniconda3/envs/alishadev/lib/python3.9/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


hidden units 8
dropout rate 0.3
learning rate 0.001
weight decay 0.0001
batch size 16
num epochs 100
hidden units 8
dropout rate 0.3
learning rate 0.001
weight decay 0.0001
batch size 16
num epochs 200
hidden units 8
dropout rate 0.3
learning rate 0.001
weight decay 0.0001
batch size 16
num epochs 300
hidden units 8
dropout rate 0.3
learning rate 0.001
weight decay 0.0001
batch size 32
num epochs 100
hidden units 8
dropout rate 0.3
learning rate 0.001
weight decay 0.0001
batch size 32
num epochs 200
hidden units 8
dropout rate 0.3
learning rate 0.001
weight decay 0.0001
batch size 32
num epochs 300
hidden units 8
dropout rate 0.3
learning rate 0.001
weight decay 0.0001
batch size 64
num epochs 100
hidden units 8
dropout rate 0.3
learning rate 0.001
weight decay 0.0001
batch size 64
num epochs 200
hidden units 8
dropout rate 0.3
learning rate 0.001
weight decay 0.0001
batch size 64
num epochs 300
hidden units 8
dropout rate 0.3
learning rate 0.001
weight decay 0.0005
batch size 16
num e

hidden units 8
dropout rate 0.5
learning rate 0.001
weight decay 0.0001
batch size 16
num epochs 300
hidden units 8
dropout rate 0.5
learning rate 0.001
weight decay 0.0001
batch size 32
num epochs 100
hidden units 8
dropout rate 0.5
learning rate 0.001
weight decay 0.0001
batch size 32
num epochs 200
hidden units 8
dropout rate 0.5
learning rate 0.001
weight decay 0.0001
batch size 32
num epochs 300
hidden units 8
dropout rate 0.5
learning rate 0.001
weight decay 0.0001
batch size 64
num epochs 100
hidden units 8
dropout rate 0.5
learning rate 0.001
weight decay 0.0001
batch size 64
num epochs 200
hidden units 8
dropout rate 0.5
learning rate 0.001
weight decay 0.0001
batch size 64
num epochs 300
hidden units 8
dropout rate 0.5
learning rate 0.001
weight decay 0.0005
batch size 16
num epochs 100
hidden units 8
dropout rate 0.5
learning rate 0.001
weight decay 0.0005
batch size 16
num epochs 200
hidden units 8
dropout rate 0.5
learning rate 0.001
weight decay 0.0005
batch size 16
num e

hidden units 8
dropout rate 0.7
learning rate 0.001
weight decay 0.0001
batch size 32
num epochs 200
hidden units 8
dropout rate 0.7
learning rate 0.001
weight decay 0.0001
batch size 32
num epochs 300
hidden units 8
dropout rate 0.7
learning rate 0.001
weight decay 0.0001
batch size 64
num epochs 100
hidden units 8
dropout rate 0.7
learning rate 0.001
weight decay 0.0001
batch size 64
num epochs 200
hidden units 8
dropout rate 0.7
learning rate 0.001
weight decay 0.0001
batch size 64
num epochs 300
hidden units 8
dropout rate 0.7
learning rate 0.001
weight decay 0.0005
batch size 16
num epochs 100
hidden units 8
dropout rate 0.7
learning rate 0.001
weight decay 0.0005
batch size 16
num epochs 200
hidden units 8
dropout rate 0.7
learning rate 0.001
weight decay 0.0005
batch size 16
num epochs 300
hidden units 8
dropout rate 0.7
learning rate 0.001
weight decay 0.0005
batch size 32
num epochs 100
hidden units 8
dropout rate 0.7
learning rate 0.001
weight decay 0.0005
batch size 32
num e

hidden units 16
dropout rate 0.3
learning rate 0.001
weight decay 0.0001
batch size 64
num epochs 100
hidden units 16
dropout rate 0.3
learning rate 0.001
weight decay 0.0001
batch size 64
num epochs 200
hidden units 16
dropout rate 0.3
learning rate 0.001
weight decay 0.0001
batch size 64
num epochs 300
hidden units 16
dropout rate 0.3
learning rate 0.001
weight decay 0.0005
batch size 16
num epochs 100
hidden units 16
dropout rate 0.3
learning rate 0.001
weight decay 0.0005
batch size 16
num epochs 200
hidden units 16
dropout rate 0.3
learning rate 0.001
weight decay 0.0005
batch size 16
num epochs 300
hidden units 16
dropout rate 0.3
learning rate 0.001
weight decay 0.0005
batch size 32
num epochs 100
hidden units 16
dropout rate 0.3
learning rate 0.001
weight decay 0.0005
batch size 32
num epochs 200
hidden units 16
dropout rate 0.3
learning rate 0.001
weight decay 0.0005
batch size 32
num epochs 300
hidden units 16
dropout rate 0.3
learning rate 0.001
weight decay 0.0005
batch siz

...

hidden units 16
dropout rate 0.7
learning rate 0.001
weight decay 0.0001
batch size 64
num epochs 300
hidden units 16
dropout rate 0.7
learning rate 0.001
weight decay 0.0005
batch size 16
num epochs 100
hidden units 16
dropout rate 0.7
learning rate 0.001
weight decay 0.0005
batch size 16
num epochs 200
hidden units 16
dropout rate 0.7
learning rate 0.001
weight decay 0.0005
batch size 16
num epochs 300
hidden units 16
dropout rate 0.7
learning rate 0.001
weight decay 0.0005
batch size 32
num epochs 100
hidden units 16
dropout rate 0.7
learning rate 0.001
weight decay 0.0005
batch size 32
num epochs 200
hidden units 16
dropout rate 0.7
learning rate 0.001
weight decay 0.0005
batch size 32
num epochs 300
hidden units 16
dropout rate 0.7
learning rate 0.001
weight decay 0.0005
batch size 64
num epochs 100
hidden units 16
dropout rate 0.7
learning rate 0.001
weight decay 0.0005
batch size 64
num epochs 200
hidden units 16
dropout rate 0.7
learning rate 0.001
weight decay 0.0005
batch siz

hidden units 32
dropout rate 0.3
learning rate 0.001
weight decay 0.0005
batch size 16
num epochs 100
hidden units 32
dropout rate 0.3
learning rate 0.001
weight decay 0.0005
batch size 16
num epochs 200
hidden units 32
dropout rate 0.3
learning rate 0.001
weight decay 0.0005
batch size 16
num epochs 300
hidden units 32
dropout rate 0.3
learning rate 0.001
weight decay 0.0005
batch size 32
num epochs 100
hidden units 32
dropout rate 0.3
learning rate 0.001
weight decay 0.0005
batch size 32
num epochs 200
hidden units 32
dropout rate 0.3
learning rate 0.001
weight decay 0.0005
batch size 32
num epochs 300
hidden units 32
dropout rate 0.3
learning rate 0.001
weight decay 0.0005
batch size 64
num epochs 100
hidden units 32
dropout rate 0.3
learning rate 0.001
weight decay 0.0005
batch size 64
num epochs 200
hidden units 32
dropout rate 0.3
learning rate 0.001
weight decay 0.0005
batch size 64
num epochs 300
hidden units 32
dropout rate 0.3
learning rate 0.001
weight decay 0.001
batch size

hidden units 32
dropout rate 0.5
learning rate 0.001
weight decay 0.0005
batch size 16
num epochs 200
hidden units 32
dropout rate 0.5
learning rate 0.001
weight decay 0.0005
batch size 16
num epochs 300
hidden units 32
dropout rate 0.5
learning rate 0.001
weight decay 0.0005
batch size 32
num epochs 100
hidden units 32
dropout rate 0.5
learning rate 0.001
weight decay 0.0005
batch size 32
num epochs 200
hidden units 32
dropout rate 0.5
learning rate 0.001
weight decay 0.0005
batch size 32
num epochs 300
hidden units 32
dropout rate 0.5
learning rate 0.001
weight decay 0.0005
batch size 64
num epochs 100
hidden units 32
dropout rate 0.5
learning rate 0.001
weight decay 0.0005
batch size 64
num epochs 200
hidden units 32
dropout rate 0.5
learning rate 0.001
weight decay 0.0005
batch size 64
num epochs 300
hidden units 32
dropout rate 0.5
learning rate 0.001
weight decay 0.001
batch size 16
num epochs 100
hidden units 32
dropout rate 0.5
learning rate 0.001
weight decay 0.001
batch size 

hidden units 32
dropout rate 0.7
learning rate 0.001
weight decay 0.0005
batch size 16
num epochs 300
hidden units 32
dropout rate 0.7
learning rate 0.001
weight decay 0.0005
batch size 32
num epochs 100
hidden units 32
dropout rate 0.7
learning rate 0.001
weight decay 0.0005
batch size 32
num epochs 200
hidden units 32
dropout rate 0.7
learning rate 0.001
weight decay 0.0005
batch size 32
num epochs 300
hidden units 32
dropout rate 0.7
learning rate 0.001
weight decay 0.0005
batch size 64
num epochs 100
hidden units 32
dropout rate 0.7
learning rate 0.001
weight decay 0.0005
batch size 64
num epochs 200
hidden units 32
dropout rate 0.7
learning rate 0.001
weight decay 0.0005
batch size 64
num epochs 300
hidden units 32
dropout rate 0.7
learning rate 0.001
weight decay 0.001
batch size 16
num epochs 100
hidden units 32
dropout rate 0.7
learning rate 0.001
weight decay 0.001
batch size 16
num epochs 200
hidden units 32
dropout rate 0.7
learning rate 0.001
weight decay 0.001
batch size 1

Epoch: 066, Loss: 79.71484
Epoch: 067, Loss: 74.35325
Epoch: 068, Loss: 71.72349
Epoch: 069, Loss: 69.03064
Epoch: 070, Loss: 57.00100
Epoch: 071, Loss: 60.98090
Epoch: 072, Loss: 45.65240
Epoch: 073, Loss: 45.52892
Epoch: 074, Loss: 49.86869
Epoch: 075, Loss: 47.76433
Epoch: 076, Loss: 32.23641
Epoch: 077, Loss: 33.55465
Epoch: 078, Loss: 24.34041
Epoch: 079, Loss: 28.97550
Epoch: 080, Loss: 28.72981
Epoch: 081, Loss: 19.90781
Epoch: 082, Loss: 19.72640
Epoch: 083, Loss: 19.14322
Epoch: 084, Loss: 17.93216
Epoch: 085, Loss: 20.39457
Epoch: 086, Loss: 16.32152
Epoch: 087, Loss: 15.04459
Epoch: 088, Loss: 12.14665
Epoch: 089, Loss: 10.84565
Epoch: 090, Loss: 10.08311
Epoch: 091, Loss: 7.82386
Epoch: 092, Loss: 7.76858
Epoch: 093, Loss: 8.59700
Epoch: 094, Loss: 7.44637
Epoch: 095, Loss: 5.50371
Epoch: 096, Loss: 6.01798
Epoch: 097, Loss: 4.86784
Epoch: 098, Loss: 5.02503
Epoch: 099, Loss: 4.73937
Epoch: 100, Loss: 4.06784
Epoch: 101, Loss: 6.39306
Epoch: 102, Loss: 3.51414
Epoch: 103, L

In [25]:
accuracy

0.23039215686274508